# A description of the problem

In Manhattan, New York,if someone is looking to open a restaurant, where would you recommend that they open it?

# A discussion of the background

Which variables should we control to locate?

What factors do we think will affect the location?

# A description of the data

There are three data sources here.


The first data source is that average population of Manhattan's various neighborhoods


Then clean up the data and export the CSV file,it called 'aop_m.csv'



In [34]:
manhattan_p = pd.read_csv('Manhattan_Population.csv')
manhattan_p.head()

,Neighborhood,Population
0,Marble Hill,9481
1,Chinatown,47840
2,Washington Heights,201600
3,Inwood,9792
4,Hamilton Heights,47530


The second source of data is a list of location of each neighborhood in New York City

I downloaded the files and placed it on the folder,it called 'newyork_data.json'

In [4]:
import json # library to handle JSON files
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

The third data source is to use the foursquare to get the venue near each neighborhood.

# How it will be used to solve the problem.

Explore the Manhattan community with the Foursquare API. 


Use the exploration function to get the restaurants in each community (peer competitiveness) and calculate the competitiveness indicators of each region (the lower the better). 


Use the average demographic data for each neighborhood in Manhattan to calculate population density indicators for each community (the higher the better). 


Then calculate the comprehensive index 


the comprehensive index = competitiveness index * corresponding weight + population density indicator * corresponding weight. 


Finally, use the Folium library to visualize which neighborhoods in Manhattan are most likely to be catering sites.

# Explore Dataset

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [7]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Notice how all the relevant data is in the features key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [8]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [9]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

### Tranform the data into a pandas dataframe

The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. So let's start by creating an empty dataframe.

In [10]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [11]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.

In [12]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [13]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


However, we only need to analyze Manhattan, So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [28]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()
manhattan_data['Neighborhood']

0             Marble Hill
1               Chinatown
2      Washington Heights
3                  Inwood
4        Hamilton Heights
5          Manhattanville
6          Central Harlem
7             East Harlem
8         Upper East Side
9               Yorkville
10             Lenox Hill
11       Roosevelt Island
12        Upper West Side
13         Lincoln Square
14                Clinton
15                Midtown
16            Murray Hill
17                Chelsea
18      Greenwich Village
19           East Village
20        Lower East Side
21                Tribeca
22           Little Italy
23                   Soho
24           West Village
25       Manhattan Valley
26    Morningside Heights
27               Gramercy
28      Battery Park City
29     Financial District
30          Carnegie Hill
31                   Noho
32           Civic Center
33          Midtown South
34           Sutton Place
35             Turtle Bay
36             Tudor City
37        Stuyvesant Town
38          

In [15]:
print('The dataframe has {} neighborhoods.'.format(
       manhattan_data.shape[0]
    )
)

The dataframe has 40 neighborhoods.


### Use geopy library to get the latitude and longitude values of Manhattan.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [21]:
address = 'Manhattan,New York'

geolocator = Nominatim(user_agent="m_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


### Create a map of Manhattan with neighborhoods superimposed on top.

In [241]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

### Define Foursquare Credentials and Version

In [24]:
CLIENT_ID = 'IK5VOW5SYQON1SG500HT1ECBSFD5NA1CKERZ5YLSG5LCNDFE' # your Foursquare ID
CLIENT_SECRET = 'QZ1BDKWEU0JSEPKHQBRCEGZWWRHP241TEIB2ACRVKBYY14KV' # your Foursquare Secret
VERSION = '20190605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IK5VOW5SYQON1SG500HT1ECBSFD5NA1CKERZ5YLSG5LCNDFE
CLIENT_SECRET:QZ1BDKWEU0JSEPKHQBRCEGZWWRHP241TEIB2ACRVKBYY14KV


# Explore Neighborhoods in Manhattan

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [38]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.

Get the top 100 venues that are in all the neighborhood within a radius of 500 meters.

In [66]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Let's check the size of the resulting dataframe

In [67]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3324, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


Let's find out how many unique categories can be curated from all the returned venues

In [68]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 341 uniques categories.


In [69]:
manhattan_venues['Venue Category'].unique()

array(['Pizza Place', 'Yoga Studio', 'Diner', 'Coffee Shop', 'Donut Shop',
       'Gym', 'Tennis Stadium', 'Seafood Restaurant', 'Department Store',
       'Pharmacy', 'Supplement Shop', 'Video Game Store',
       'Discount Store', 'Steakhouse', 'Ice Cream Shop', 'Bank',
       'American Restaurant', 'Sandwich Place', 'Clothing Store',
       'Shopping Mall', 'Deli / Bodega', 'Spa', 'Hotel',
       'Greek Restaurant', 'Cocktail Bar', 'Chinese Restaurant',
       'English Restaurant', 'Museum', 'Bike Shop', 'Garden Center',
       'Noodle House', 'Roof Deck', 'Tea Room', 'Salon / Barbershop',
       'New American Restaurant', 'Indie Movie Theater',
       'Bubble Tea Shop', 'Korean Restaurant', 'Bakery',
       'Spanish Restaurant', 'Hotpot Restaurant', 'Historic Site',
       'Sake Bar', 'Furniture / Home Store', 'Bar', 'Record Shop',
       'Asian Restaurant', 'Malay Restaurant', 'Organic Grocery',
       'Dumpling Restaurant', "Men's Store", 'Italian Restaurant',
       'Cosmetics Sh

According to the catalog, we only need to keep the catalog containing the keywords.

Keywords : 'Diner', 'Pizza', 'Restaurant', 'Sandwich', 'Noodle', 'Burger', 'BBQ', 'Hot Dog', 'Soup'

In [103]:
keywords=('Diner', 'Pizza', 'Restaurant', 'Sandwich', 'Noodle', 'Burger', 'BBQ', 'Hot Dog', 'Soup')

diner_venues = manhattan_venues.loc[manhattan_venues['Venue Category'].str.contains('Diner')]
pizza_venues = manhattan_venues.loc[manhattan_venues['Venue Category'].str.contains('Pizza')]
restaurant_venues = manhattan_venues.loc[manhattan_venues['Venue Category'].str.contains('Restaurant')]
sandwich_venues = manhattan_venues.loc[manhattan_venues['Venue Category'].str.contains('Sandwich')]
noodle_venues = manhattan_venues.loc[manhattan_venues['Venue Category'].str.contains('Noodle')]
burger_venues = manhattan_venues.loc[manhattan_venues['Venue Category'].str.contains('Burger')]
bbq_venues = manhattan_venues.loc[manhattan_venues['Venue Category'].str.contains('BBQ')]
hd_venues = manhattan_venues.loc[manhattan_venues['Venue Category'].str.contains('Hot Dog')]
soup_venues = manhattan_venues.loc[manhattan_venues['Venue Category'].str.contains('Soup')]

diner_list = list(diner_venues['Venue Category'])
pizza_list = list(pizza_venues['Venue Category'])
restaurant_list = list(restaurant_venues['Venue Category'])
sandwich_list = list(sandwich_venues['Venue Category'])
noodle_list = list(noodle_venues['Venue Category'])
burger_list = list(burger_venues['Venue Category'])
bbq_list = list(bbq_venues['Venue Category'])
hd_list = list(hd_venues['Venue Category'])
soup_list = list(soup_venues['Venue Category'])

v_list = diner_list + pizza_list+ restaurant_list+ sandwich_list+ noodle_list+burger_list +bbq_list + hd_list+ soup_list
v_list = set(v_list)

for key in v_list:
    m_diner = manhattan_venues[manhattan_venues['Venue Category'].isin(v_list)]
    
m_diner.shape


(1098, 7)

In [104]:
m_diner = m_diner.reset_index(drop=True)
m_diner

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.910660,Tibbett Diner,40.880404,-73.908937,Diner
2,Marble Hill,40.876551,-73.910660,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant
3,Marble Hill,40.876551,-73.910660,Boston Market,40.877430,-73.905412,American Restaurant
4,Marble Hill,40.876551,-73.910660,SUBWAY,40.878493,-73.905385,Sandwich Place
5,Marble Hill,40.876551,-73.910660,Subway,40.877720,-73.905380,Sandwich Place
6,Chinatown,40.715618,-73.994279,Kiki's,40.714476,-73.992036,Greek Restaurant
7,Chinatown,40.715618,-73.994279,Spicy Village,40.717010,-73.993530,Chinese Restaurant
8,Chinatown,40.715618,-73.994279,The Fat Radish,40.715323,-73.991950,English Restaurant
9,Chinatown,40.715618,-73.994279,Scarr's Pizza,40.715335,-73.991649,Pizza Place


# Analyze Each Neighborhood

In [119]:
# one hot encoding
manhattan_onehot = pd.get_dummies(m_diner[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = m_diner['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Brazilian Restaurant,Burger Joint,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Diner,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hot Dog Joint,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Pizza Place,Ramen Restaurant,Restaurant,Russian Restaurant,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shanghai Restaurant,Soba Restaurant,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [121]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Brazilian Restaurant,Burger Joint,Cambodian Restaurant,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Chinese Restaurant,Cuban Restaurant,Czech Restaurant,Dim Sum Restaurant,Diner,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hot Dog Joint,Hotpot Restaurant,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Persian Restaurant,Peruvian Restaurant,Pizza Place,Ramen Restaurant,Restaurant,Russian Restaurant,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shanghai Restaurant,Soba Restaurant,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant
0,Battery Park City,0.000000,0.0000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.117647,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.176471,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117647,0.000000,0.000000,0.00000,0.117647,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Carnegie Hill,0.000000,0.0000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.066667,0.000000,0.066667,0.000000,0.100000,0.000000,0.00000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.033333,0.033333,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.066667
2,Central Harlem,0.000000,0.1875,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.00000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

Let's print each neighborhood along with the top 5 most common kinds of diner

In [125]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['kinds of diner','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
       kinds of diner  freq
0  Italian Restaurant  0.18
1         Pizza Place  0.12
2           BBQ Joint  0.12
3        Burger Joint  0.12
4      Sandwich Place  0.12


----Carnegie Hill----
          kinds of diner  freq
0            Pizza Place  0.20
1    Japanese Restaurant  0.10
2      French Restaurant  0.10
3  Vietnamese Restaurant  0.07
4     Italian Restaurant  0.07


----Central Harlem----
        kinds of diner  freq
0   African Restaurant  0.19
1   Seafood Restaurant  0.12
2   Chinese Restaurant  0.12
3  American Restaurant  0.12
4    French Restaurant  0.12


----Chelsea----
        kinds of diner  freq
0   Italian Restaurant  0.19
1  American Restaurant  0.11
2   Seafood Restaurant  0.11
3    French Restaurant  0.07
4     Tapas Restaurant  0.07


----Chinatown----
          kinds of diner  freq
0     Chinese Restaurant  0.19
1  Vietnamese Restaurant  0.08
2    American Restaurant  0.08
3    Dumpling Restaurant  0.06
4   Cantonese Restaurant  0.04

### Calculate the number of restaurants in each neighborhood

In [136]:
m_diner_counts = m_diner.groupby('Neighborhood').count()
m_diner_counts['Venue']

Neighborhood
Battery Park City      17
Carnegie Hill          30
Central Harlem         16
Chelsea                27
Chinatown              48
Civic Center           27
Clinton                28
East Harlem            18
East Village           46
Financial District     26
Flatiron               32
Gramercy               29
Greenwich Village      48
Hamilton Heights       25
Hudson Yards           24
Inwood                 22
Lenox Hill             35
Lincoln Square         14
Little Italy           29
Lower East Side        21
Manhattan Valley       27
Manhattanville         18
Marble Hill             6
Midtown                24
Midtown South          30
Morningside Heights    14
Murray Hill            43
Noho                   38
Roosevelt Island        4
Soho                   21
Stuyvesant Town         1
Sutton Place           33
Tribeca                28
Tudor City             39
Turtle Bay             44
Upper East Side        29
Upper West Side        37
Washington Heights     26

### Calculate peer competition metrics for each neighborhood

How many restaurants are there in Manhattan?

In [137]:
diner_sum = m_diner_counts['Venue'].sum()
diner_sum

1098

Calculate peer competition metrics for each neighborhood

In [209]:
m_diner_counts['Venue_Frq'] = m_diner_counts['Venue'].map(lambda x: x/diner_sum).round(4) 
m_diner_counts['Venue_Frq']

Neighborhood
Battery Park City      0.0155
Carnegie Hill          0.0273
Central Harlem         0.0146
Chelsea                0.0246
Chinatown              0.0437
Civic Center           0.0246
Clinton                0.0255
East Harlem            0.0164
East Village           0.0419
Financial District     0.0237
Flatiron               0.0291
Gramercy               0.0264
Greenwich Village      0.0437
Hamilton Heights       0.0228
Hudson Yards           0.0219
Inwood                 0.0200
Lenox Hill             0.0319
Lincoln Square         0.0128
Little Italy           0.0264
Lower East Side        0.0191
Manhattan Valley       0.0246
Manhattanville         0.0164
Marble Hill            0.0055
Midtown                0.0219
Midtown South          0.0273
Morningside Heights    0.0128
Murray Hill            0.0392
Noho                   0.0346
Roosevelt Island       0.0036
Soho                   0.0191
Stuyvesant Town        0.0009
Sutton Place           0.0301
Tribeca                0.02

### Calculate the population density index of each neighborhood

In [210]:
manhattan_p = pd.read_csv('Manhattan_Population.csv')
manhattan_p.head()

,Neighborhood,Population
0,Marble Hill,9481
1,Chinatown,47840
2,Washington Heights,201600
3,Inwood,9792
4,Hamilton Heights,47530


In [211]:
m_p = manhattan_p.set_index('Neighborhood')
m_p_sum = m_p['Population'].sum()
m_p_sum

1962259

In [212]:
m_p['Population_Frq'] = m_p['Population'].map(lambda x: x/m_p_sum).round(4) 
m_p['Population_Frq']

Neighborhood
Marble Hill            0.0048
Chinatown              0.0244
Washington Heights     0.1027
Inwood                 0.0050
Hamilton Heights       0.0242
Manhattanville         0.0207
Central Harlem         0.0593
East Harlem            0.0591
Upper East Side        0.0633
Yorkville              0.0099
Lenox Hill             0.0455
Roosevelt Island       0.0059
Upper West Side        0.1094
Lincoln Square         0.0208
Clinton                0.0130
Midtown                0.0534
Murray Hill            0.0055
Chelsea                0.0038
Greenwich Village      0.0159
East Village           0.0323
Lower East Side        0.0372
Tribeca                0.0088
Little Italy           0.0025
Soho                   0.0100
West Village           0.0162
Manhattan Valley       0.0481
Morningside Heights    0.0162
Gramercy               0.0143
Battery Park City      0.0047
Financial District     0.0294
Carnegie Hill          0.0246
Noho                   0.0020
Civic Center           0.00

### Calculate the comprehensive indicators of each neighborhood

In [213]:
m_c = pd.merge(m_p, m_diner_counts, on='Neighborhood', how='left')
m_c.drop(m_c.columns[[0, 2, 3, 4, 5, 6, 7]], axis=1, inplace=True)
m_c

,Population_Frq,Venue_Frq
Neighborhood,,
Marble Hill,0.0048,0.0055
Chinatown,0.0244,0.0437
Washington Heights,0.1027,0.0237
Inwood,0.0050,0.0200
Hamilton Heights,0.0242,0.0228
Manhattanville,0.0207,0.0164
Central Harlem,0.0593,0.0146
East Harlem,0.0591,0.0164
Upper East Side,0.0633,0.0264


According to experience, population density is often important for the catering industry.


So I set the weight of the population density indicator to 0.8, and the weight of the peer competition indicator to 0.2.

In [214]:
W_p = 0.8
W_v = 0.2
m_c['C_Frq'] = m_c.apply(lambda x: W_p *x['Population_Frq'] + W_v * x['Venue_Frq'], axis=1).round(4) 
m_c

,Population_Frq,Venue_Frq,C_Frq
Neighborhood,,,
Marble Hill,0.0048,0.0055,0.0049
Chinatown,0.0244,0.0437,0.0283
Washington Heights,0.1027,0.0237,0.0869
Inwood,0.0050,0.0200,0.0080
Hamilton Heights,0.0242,0.0228,0.0239
Manhattanville,0.0207,0.0164,0.0198
Central Harlem,0.0593,0.0146,0.0504
East Harlem,0.0591,0.0164,0.0506
Upper East Side,0.0633,0.0264,0.0559


Get a comprehensive indicator for each neighborhood

In [215]:
m_c['C_Frq']

Neighborhood
Marble Hill            0.0049
Chinatown              0.0283
Washington Heights     0.0869
Inwood                 0.0080
Hamilton Heights       0.0239
Manhattanville         0.0198
Central Harlem         0.0504
East Harlem            0.0506
Upper East Side        0.0559
Yorkville              0.0150
Lenox Hill             0.0428
Roosevelt Island       0.0054
Upper West Side        0.0943
Lincoln Square         0.0192
Clinton                0.0155
Midtown                0.0471
Murray Hill            0.0122
Chelsea                0.0080
Greenwich Village      0.0215
East Village           0.0342
Lower East Side        0.0336
Tribeca                0.0121
Little Italy           0.0073
Soho                   0.0118
West Village           0.0193
Manhattan Valley       0.0434
Morningside Heights    0.0155
Gramercy               0.0167
Battery Park City      0.0069
Financial District     0.0283
Carnegie Hill          0.0251
Noho                   0.0085
Civic Center           0.00

### Reconstruct geographic information form

In [216]:
m_gif = pd.merge(m_c, manhattan_data, on='Neighborhood', how='left')
m_gif = m_gif.drop(['Borough'],axis=1)
m_gif

,Neighborhood,Population_Frq,Venue_Frq,C_Frq,Latitude,Longitude
0,Marble Hill,0.0048,0.0055,0.0049,40.876551,-73.910660
1,Chinatown,0.0244,0.0437,0.0283,40.715618,-73.994279
2,Washington Heights,0.1027,0.0237,0.0869,40.851903,-73.936900
3,Inwood,0.0050,0.0200,0.0080,40.867684,-73.921210
4,Hamilton Heights,0.0242,0.0228,0.0239,40.823604,-73.949688
5,Manhattanville,0.0207,0.0164,0.0198,40.816934,-73.957385
6,Central Harlem,0.0593,0.0146,0.0504,40.815976,-73.943211
7,East Harlem,0.0591,0.0164,0.0506,40.792249,-73.944182
8,Upper East Side,0.0633,0.0264,0.0559,40.775639,-73.960508
9,Yorkville,0.0099,0.0355,0.0150,40.775930,-73.947118


Let's visualize the resulting

In [240]:
from folium.plugins import HeatMap

num =40
h_lat = np.array(m_gif['Latitude'][0:num])
h_lon = np.array(m_gif['Longitude'][0:num])
h_c = np.array(m_gif['C_Frq'][0:num]*1000,dtype=float)


hot_data =  [[h_lat[i],h_lon[i],h_c[i]] for i in range(num)]

# create map of Manhattan using latitude and longitude values
map_hot_manhattan = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood, c_f, p_f, v_f in zip(m_gif['Latitude'], m_gif['Longitude'], m_gif['Neighborhood'],m_gif['C_Frq'],m_gif['Population_Frq'],m_gif['Venue_Frq']):
    label = '{},C_Frq:{}'.format(neighborhood,c_f)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hot_manhattan)  
    HeatMap(hot_data).add_to(map_hot_manhattan)
map_hot_manhattan

Where the color is redder, the higher the composite index

# Results

Based on the heat map generated by the observation, we can find that the Upper West Side has the highest comprehensive index. Through the dateframe of m_gif, we can see that the population density index of this place is the highest. Although the peer competition index is relatively high, the weight of the peer competition index is relatively low, so the overall index of the Upper West Side can still reach a high level.

In Manhattan, New York,if someone is looking to open a restaurant, where would you recommend that they open it?

Until now, we solved this problem.

I would recommend someone to open a restaurant in Upper West Side or Washington Heights.

# Discuss

If you want to improve this idea further, I have a few suggestions.

1.By adding other influencing factors such as road traffic, regional tastes, local average income, etc.

2.On the basis of the original, by modifying the weight of each influencing factor to get the most suitable answer

# Conclusion

I would recommend someone to open a restaurant in Upper West Side or Washington Heights.